# Importing the libraries

In [ ]:
from env import SimpleGridWorld

import numpy as np

# Initialising the Environment

In [ ]:
envSize = 15
env = SimpleGridWorld(size=envSize, start=0, end = envSize -1)

In [ ]:
state, target = env.reset()
env.render()

# Define the Q-value table

In [ ]:
action_values = np.full(shape=(envSize,2),fill_value=-100)
action_values[target,:] = 0.

# Define the target policy

In [ ]:
def target_policy(state):
    av = action_values[state]
    return np.random.choice(np.flatnonzero(av == av.max()))

# Define the explaratory policy

In [ ]:
def explaratory_policy(state, epsilon = 0.2):
    if np.random.random() < epsilon:
        return env.action_space.sample()
    else: 
        av = action_values[state]
        return np.random.choice(np.flatnonzero(av == av.max()))

# Implement the algorithm

In [ ]:
def off_policy_mc_control(acton_values, target_policy, explaratory_policy, episodes = 10000, gamma = 0.99, epsilon = 0.2):
    csa = np.zeros((envSize, 2))

    for episode in range(1, episodes + 1):
        G, W = 0,1
        state, target = env.reset()
        done = False
        transitions = []

        while not done:
            action = explaratory_policy(state, epsilon)
            next_state, reward, done = env.step(action)
            transitions.append([state, action, reward])
            state = next_state

        for state_t, action_t, reward_t in reversed(transitions):
            G = reward_t + gamma * G
            csa[state_t][action_t] += W
            qsa = action_values[state_t][action_t]
            action_values[state_t][action_t] += (W / csa[state_t][action_t])*(G - qsa)

            if action_t != target_policy(state_t):
                break

            W = W * 1/ (1-epsilon + epsilon / 4)
        
        print(episode)

In [ ]:
off_policy_mc_control(action_values, target_policy, explaratory_policy)

# Testing the algorithm

In [ ]:
env.test_agent(target_policy, 3)